In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import requests
from selenium.common.exceptions import NoSuchElementException ,StaleElementReferenceException

In [2]:
url = 'https://www.cotodigital3.com.ar'

#this func takes the first url all with Beatiful Soup and the names, categories
def take_url():
    #taking the url
    r = requests.get(url)


    soup =BeautifulSoup(r.text,'html.parser')
    #selecting the tags
    classes=soup.find_all('ul',class_='sub_category')
    
    #looking at the href
    links= [i.find('a')['href'] for i in classes]

    #names of categories
    names=[i.a.text   for i in classes]
    #taking out the spaces and extra symbols
    names_categories = [i.replace(' (+)','').strip() for i in names]
                               
    return links ,names_categories
    


#this variable holds the function of categories and first links. This links have to iterate to another pages
first_step = take_url()

In [3]:
first_step[1]

['Golosinas',
 'Panaderia',
 'Snacks',
 'Cereales',
 'Endulzantes',
 'Aderezos y salsas',
 'Infusiones',
 'Conservas',
 'Encurtidos',
 'Harinas',
 'Mermeladas y dulces',
 'Salsas y puré de tomate',
 'Aceites y condimentos',
 'Alimento de bebés y niños',
 'Arroz y legumbres',
 'Especias',
 'Pasta seca, lista y rellenas',
 'Polvo para postres y reposteria',
 'Sopas, caldos, puré y saborizantes',
 'Rebozador y pan rallado',
 'Leche en polvo',
 'Suplementos dietarios',
 'Productos orgánicos',
 'Huevos de pascua',
 'Bebidas sin alcohol',
 'Bebidas con alcohol',
 'Lácteos',
 'Fiambres',
 'Quesos',
 'Carniceria',
 'Aves',
 'Pastas frescas y tapas',
 'Comidas elaboradas',
 'Frutas y verduras',
 'Pescaderia',
 'Huevos',
 'Helados y postres',
 'Comidas congeladas',
 'Hamburguesas y milanesas',
 'Vegetales congelados',
 'Papas congeladas / fritas',
 'Frutas congeladas',
 'Nuggets, patitas y bocaditos',
 'Pescaderia',
 'Lavado',
 'Papeles',
 'Insecticidas',
 'Calzado',
 'Accesorios de limpieza',
 

In [4]:
first_step[0]

['/sitios/cdigi/browse/catalogo-almac%C3%A9n-golosinas/_/N-1y5dh9i',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-panaderia/_/N-s3bf1a',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-snacks/_/N-10kzbyj',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-cereales/_/N-ukd5id',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-endulzantes/_/N-1rtbab6',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-aderezos-y-salsas/_/N-rv0frc',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-infusiones/_/N-dw58vw',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-conservas/_/N-1t4efca',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-encurtidos/_/N-12rkdi1',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-harinas/_/N-842qrm',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-mermeladas-y-dulces/_/N-mj4aa8',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-salsas-y-pur%C3%A9-de-tomate/_/N-os1anu',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-aceites-y-condimentos/_/N-18r69ct',
 '/sitios/cdigi/browse/catalogo-almac%C3%A9n-alimento-de-beb%C3

In [5]:

def full_urls(partial_link,names_categories):
    PATH= 'C:\Program Files (x86)\chromedriver.exe'

    link_total = 'https://www.cotodigital3.com.ar'+partial_link

    #s = Service(PATH)

    #driver = webdriver.Chrome(service=s)
    driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
    #driver=webdriver.Chrome()
    time.sleep(2)
    driver.get(link_total)
    time.sleep(1)
    driver.maximize_window()
    time.sleep(18)
    #next = driver.find_element(By.LINK_TEXT,'Sig')
    total_urls=[]
    total_urls.append(link_total)
    while True:
            try:
                next = driver.find_element(By.LINK_TEXT,'Sig')
                time.sleep(2)
                next.click()
                pages= driver.current_url
                total_urls.append(pages)
                time.sleep(7)
            except:
                 break
                 
                 
    sig= driver.find_elements(By.CSS_SELECTOR,'a[title*="Ir a página"]')
    
    for i in range(1,len(sig)):
        try:
            sig= driver.find_elements(By.CSS_SELECTOR,'a[title*="Ir a página"]')
            time.sleep(2)
            sig[i].click()
            pages= driver.current_url
            total_urls.append(pages)
            time.sleep(7)
        except IndexError:
            sig = driver.find_elements(By.CSS_SELECTOR,'a[title*="Ir a página"]')
            time.sleep(2)
            sig[1].click()
            pages=driver.current_url
            total_urls.append(pages)
    
    
    print(f'the total urls: {len(total_urls)}')
    print(f' the name categories: {names_categories}')
    nombre=names_categories*len(total_urls)                       

    driver.quit()

    return total_urls,nombre




In [6]:
main_data= [ full_urls(link,title) for link, title in zip(first_step[0],first_step[1])]

/tmp/ipykernel_8200/2016805721.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")


ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [61]:
PATH= 'C:\Program Files (x86)\chromedriver.exe'

link_total = 'https://www.cotodigital3.com.ar/sitios/cdigi/browse/catalogo-hogar-libros/_/N-q08sox'

s = Service(PATH)

driver = webdriver.Chrome(service=s)
driver.set_window_size(1366,768)
driver.get(link_total)
time.sleep(14)
    #next = driver.find_element(By.LINK_TEXT,'Sig')
total_urls=[]
while True:
        try:
            next = driver.find_element(By.LINK_TEXT,'Sig')
            time.sleep(2)
            next.click()
            pages= driver.current_url
            total_urls.append(pages)
            time.sleep(7)
        except:
            break
                 

sig= driver.find_elements(By.CSS_SELECTOR,'a[title*="Ir a página"]')
print(len(sig))
for i in range(1,len(sig)):
    try:
        sig= driver.find_elements(By.CSS_SELECTOR,'a[title*="Ir a página"]')
        print(len(sig))
        time.sleep(2)
        sig[i].click()
        pages= driver.current_url
        total_urls.append(pages)
        time.sleep(7)
    except IndexError:
        sig = driver.find_elements(By.CSS_SELECTOR,'a[title*="Ir a página"]')
        time.sleep(2)
        sig[1].click()
        pages=driver.current_url
        total_urls.append(pages)



time.sleep(2)
# if sig==True:
#     for i in len(range(sig)):
#         print(i)
#         sig[i].click()
#         pages=driver.current_url
#         total_urls.append(pages)
#         time.sleep(7)
                           

total_urls



5
5
4
4
4


['https://www.cotodigital3.com.ar/sitios/cdigi/browse/catalogo-hogar-libros/_/N-q08sox?Nf=product.startDate%7CLTEQ+1.6782336E12%7C%7Cproduct.endDate%7CGTEQ+1.6782336E12&No=48&Nr=AND%28product.sDisp_200%3A1004%2Cproduct.language%3Aespa%C3%B1ol%2COR%28product.siteId%3ACotoDigital%29%29&Nrpp=48',
 'https://www.cotodigital3.com.ar/sitios/cdigi/browse/catalogo-hogar-libros/_/N-q08sox?Nf=product.startDate%7CLTEQ+1.6782336E12%7C%7Cproduct.endDate%7CGTEQ+1.6782336E12&No=144&Nr=AND%28product.language%3Aespa%C3%B1ol%2Cproduct.sDisp_200%3A1004%2COR%28product.siteId%3ACotoDigital%29%29&Nrpp=48',
 'https://www.cotodigital3.com.ar/sitios/cdigi/browse/catalogo-hogar-libros/_/N-q08sox?Nf=product.startDate%7CLTEQ+1.6782336E12%7C%7Cproduct.endDate%7CGTEQ+1.6782336E12&No=192&Nr=AND%28product.sDisp_200%3A1004%2Cproduct.language%3Aespa%C3%B1ol%2COR%28product.siteId%3ACotoDigital%29%29&Nrpp=48',
 'https://www.cotodigital3.com.ar/sitios/cdigi/browse/catalogo-hogar-libros/_/N-q08sox?Nf=product.startDate%7CLTE

In [48]:
len(total_urls)*'perro'

'perroperroperroperroperroperroperroperroperroperro'

In [118]:

#url ='https://www.cotodigital3.com.ar/sitios/cdigi/browse/ofertas-exclusivas/_/N-1nx2iz5'

#url= 'https://www.cotodigital3.com.ar/sitios/cdigi/browse/catalogo-almac%C3%A9n-golosinas/_/N-1y5dh9i'


r = requests.get(url)


soup =BeautifulSoup(r.text,'html.parser')

def take_product():
    products = soup.find_all('li',class_='clearfix')

    url_product=[]
    description_product=[]
    precio_contado=[]
    promocion=[]
    precio_descuento=[]
    for product_description in products:

        #link product
        link_product=product_description.find('a')['href']
        url_product.append(link_product)

        #description product
        description= product_description.find('div',class_='descrip_full').text
        description_product.append(description)

        #precio por unidad
        try:
            unidad = product_description.find('div',class_='price_regular_container').text
            precio_contado.append(unidad)
        except AttributeError:
            unidad = product_description.find('span',class_='atg_store_newPrice').text
            precio_contado.append(unidad)


        #descuento
        try:
            descuento = product_description.find('span',class_='text_price_discount').text
            promocion.append(descuento)
        except AttributeError:
            promocion.append(None)

        #precio descuento
        try:
            precio_final=product_description.find('span',class_='price_discount').text
            precio_descuento.append(precio_final)
        except AttributeError:
            precio_descuento.append(None)

        

    return url_product, description_product,precio_contado,promocion, precio_descuento
    
productos = take_product()



productos

(['/sitios/cdigi/producto/-pastillas-tic-tac-menta-cja-16-grm/_/A-00072184-00072184-200',
  '/sitios/cdigi/producto/-pastillas-surtido-tic-tac-cja-16-grm/_/A-00457325-00457325-200',
  '/sitios/cdigi/producto/-pastillas-tic-tac-naranja-cja-16-grm/_/A-00072183-00072183-200',
  '/sitios/cdigi/producto/-pastillas-menta-fuerte-tic-tac-pas-16-grm/_/A-00489937-00489937-200',
  '/sitios/cdigi/producto/-chocolate-kitkat-4-fingers-x-415gr/_/A-00272306-00272306-200',
  '/sitios/cdigi/producto/-chocolate-blanco-kitkat-4-fingers-x-415gr/_/A-00474724-00474724-200',
  '/sitios/cdigi/producto/-caramelos-duro-clight-naranja-dulce-paq-20-grm/_/A-00248875-00248875-200',
  '/sitios/cdigi/producto/-caramelos-clight-frambuesa-20gr/_/A-00256933-00256933-200',
  '/sitios/cdigi/producto/-caramelos-halls-free-cherry-x-20gr/_/A-00241203-00241203-200',
  '/sitios/cdigi/producto/-oblea-de-arroz-gallo-snacks-paq-20-grm/_/A-00289874-00289874-200',
  '/sitios/cdigi/producto/-bocadito-choconuss-gallo-snack-paq-20-grm/

In [119]:
print(len(productos[0]))
print(len(productos[1]))
print(len(productos[2]))
print(len(productos[3]))
print(len(productos[4]))

48
48
48
48
48
